In [11]:
import pandas as pd
import boto3
import json

## Using accesskey and secreykey 

In [12]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh_cluster.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')


DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")



## Create clients for EC2, S3, IAM, and Redshift

In [13]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="ap-south-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )
s3 =  boto3.resource('s3',
                       region_name="ap-south-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )
iam =  boto3.client('iam',
                       region_name="ap-south-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )
redshift =  boto3.client('redshift',
                       region_name="ap-south-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

# CREATE IAM ROLE

In [14]:
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Redshift cluster demo for IAC.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    

except Exception as e:
    print(e)

1.1 Creating a new IAM Role


# ATTACH POLICY

In [15]:
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']



1.2 Attaching Policy


# CREATE REDSHIFT CLUSTER

In [16]:
try:
    response = redshift.create_cluster(        
        # TODO: add parameters for hardware
        
            NodeType=DWH_NODE_TYPE,           
            NumberOfNodes=int(DWH_NUM_NODES),
            ClusterType=DWH_CLUSTER_TYPE,

        # TODO: add parameters for identifiers & credentials
            MasterUsername=DWH_DB_USER,
            MasterUserPassword=DWH_DB_PASSWORD,
            DBName=DWH_DB,
            ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        # TODO: add parameter for role (to allow s3 access)
            IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

# CHECK THE STATUS

In [20]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
